In [37]:
import pandas as pd
import numpy as np
import random
import os
import gc
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(42) # Seed 고정

In [38]:
train = pd.read_parquet('./train.parquet')
test = pd.read_parquet('./test.parquet')
sample_submission = pd.read_csv('sample_submission.csv', index_col = 0)

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

RuntimeError: module compiled against API version 0xe but this version of numpy is 0xd

In [39]:
train = train.drop('Carrier_Code(IATA)',axis=1)
train = train.drop('Origin_Airport',axis=1)
train = train.drop('Destination_Airport',axis=1)
train = train.drop(['Origin_State','Destination_State'],axis=1)
train = train.drop(['Airline'],axis=1)

lbe = LabelEncoder()
train['Tail_Number'] = lbe.fit_transform(train['Tail_Number'])

In [40]:
train_ = pd.DataFrame()
train_ = pd.concat([train_,train[train['Delay']=='Not_Delayed']])
train_ = pd.concat([train_,train[train['Delay']=='Delayed']])
train_ = train_.dropna()
train_

lbe = LabelEncoder()
train_['Delay'] = lbe.fit_transform(train_['Delay'])
train_

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay
8,TRAIN_000008,6,13,1420.0,1550.0,0,0,10821,11057,361.0,19393.0,4083,1
13,TRAIN_000013,9,19,615.0,706.0,0,0,13158,11292,563.0,20304.0,579,1
19,TRAIN_000019,7,14,1907.0,2145.0,0,0,10529,10397,859.0,19790.0,6147,1
32,TRAIN_000032,6,20,750.0,1014.0,0,0,13930,12945,323.0,20366.0,327,1
42,TRAIN_000042,10,24,1005.0,1130.0,0,0,13930,15016,258.0,20237.0,4766,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999816,TRAIN_999816,11,6,2115.0,2348.0,0,0,13930,11076,360.0,20304.0,6171,0
999843,TRAIN_999843,6,13,1749.0,1850.0,0,0,13342,13930,67.0,20237.0,263,0
999955,TRAIN_999955,4,12,1310.0,1356.0,0,0,14747,14057,129.0,19687.0,2250,0
999963,TRAIN_999963,5,2,1759.0,1926.0,0,0,12953,11278,214.0,20452.0,5204,0


In [41]:
lbe.classes_

array(['Delayed', 'Not_Delayed'], dtype=object)

In [42]:
test_ = train[train['Delay'].isnull()]
test_ = test_.drop('Delay',axis=1)
test_ = test_.dropna()
test_

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number
1,TRAIN_000001,8,15,740.0,1024.0,0,0,13930,14869,1250.0,20304.0,310
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,11057,12953,544.0,19805.0,140
4,TRAIN_000004,1,11,900.0,1019.0,0,0,14771,10157,250.0,20304.0,556
7,TRAIN_000007,4,20,1815.0,1955.0,0,0,13930,13198,403.0,20304.0,173
9,TRAIN_000009,6,6,650.0,838.0,0,0,12992,12266,374.0,20366.0,468
...,...,...,...,...,...,...,...,...,...,...,...,...
999993,TRAIN_999993,10,17,603.0,755.0,0,0,12264,11292,1452.0,20436.0,1538
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,13930,14100,678.0,19977.0,2477
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,13796,12191,1642.0,19393.0,994
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,10693,10397,214.0,19790.0,6207


In [43]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 13 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null   float64
 4   Estimated_Arrival_Time    890960 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport_ID         1000000 non-null  int64  
 8   Destination_Airport_ID    1000000 non-null  int64  
 9   Distance                  1000000 non-null  float64
 10  Carrier_ID(DOT)           891003 non-null   float64
 11  Tail_Number               1000000 non-null  int64  
 12  Delay                     255001 non-null   object 
dtypes: float64(4), int64(7), obj

# column filling

In [44]:
target = 'Estimated_Departure_Time'
features = ['Month',
 'Day_of_Month',
 #'Estimated_Departure_Time',
 #'Estimated_Arrival_Time',
 'Cancelled',
 'Diverted',
 'Origin_Airport_ID',
 'Destination_Airport_ID',
 'Distance',
 #'Carrier_ID(DOT)',
 'Tail_Number']

In [45]:
test_r = train[train[target].isnull()]
train_r = train.drop(test_r.index)
train_r = train_r.reset_index(drop=True)

# mms = MinMaxScaler()
# train_r[target] = mms.fit_transform(train_r[[target]])

In [46]:
from catboost import CatBoostClassifier, CatBoostRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
from xgboost import XGBClassifier
from sklearn.metrics import log_loss, mean_squared_error, mean_absolute_error
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

kf = KFold(n_splits = 10, random_state = 41, shuffle = True)
models = []
val_scores = []
preds = []

params = {
'n_estimators':5000,
'max_depth':6,
'early_stopping_rounds':200,
'learning_rate':0.01,
'one_hot_max_size':3,
'bootstrap_type':"MVS",
'l2_leaf_reg':2,
'random_state':41
         }

# bundle = [CatBoostClassifier, LGBMClassifier, XGBClassifier]
# for model in bundle:
# model = CatBoostClassifier(**params)
for i, (train_index, val_index) in tqdm(enumerate(kf.split(train_r, train_r[target]))):

        # clf.fit(train_x, train_y)

        X_train, X_val = train_r[features].loc[train_index], train_r[features].loc[val_index]
        y_train, y_val = train_r[target][train_index], train_r[target][val_index]

        model = LGBMRegressor(**params)
    
        model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = 2000)

        models.append(model)
        score = mean_absolute_error(y_val, model.predict(X_val))
        print(f'''
        ***********************************
            score : {score}                           
        ***********************************
        ''')

        val_scores.append(score)

0it [00:00, ?it/s]/Users/junho/miniforge3/envs/ml_dl/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] Unknown parameter: l2_leaf_reg
[LightGBM] [Warning] Unknown parameter: bootstrap_type
[LightGBM] [Warning] Unknown parameter: one_hot_max_size
[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200
[2000]	valid_0's l2: 223723
[4000]	valid_0's l2: 220220


1it [00:48, 48.59s/it]


        ***********************************
            score : 404.13013903551894                           
        ***********************************
        
[LightGBM] [Warning] Unknown parameter: l2_leaf_reg
[LightGBM] [Warning] Unknown parameter: bootstrap_type
[LightGBM] [Warning] Unknown parameter: one_hot_max_size
[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200


/Users/junho/miniforge3/envs/ml_dl/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


...


        ***********************************
            score : 404.63805681444603                           
        ***********************************
        
[LightGBM] [Warning] Unknown parameter: l2_leaf_reg
[LightGBM] [Warning] Unknown parameter: bootstrap_type
[LightGBM] [Warning] Unknown parameter: one_hot_max_size
[LightGBM] [Warning] early_stopping_round is set=200, early_stopping_rounds=200 will be ignored. Current value: early_stopping_round=200


/Users/junho/miniforge3/envs/ml_dl/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[2000]	valid_0's l2: 223768
[4000]	valid_0's l2: 220468


10it [06:14, 37.40s/it]


        ***********************************
            score : 404.68920007027685                           
        ***********************************
        


In [50]:
cat_preds = []
for model in models[:10]:
    preds = model.predict(test_r[features])
    cat_preds.append(preds)

cat_preds

[array([1292.93085608, 1327.45239835, 1210.52498401, ..., 1242.72912028,
        1285.32847974, 1313.87668589]),
 array([1308.4813839 , 1352.35873426, 1212.63746846, ..., 1256.31998115,
        1287.03316324, 1301.2069708 ]),
 array([1305.72223927, 1361.1803405 , 1210.6297118 , ..., 1239.5500146 ,
        1271.71470284, 1311.08694497]),
 array([1321.76487076, 1393.65246826, 1207.56829742, ..., 1257.51113681,
        1268.11492942, 1304.18729826]),
 array([1301.35035719, 1389.24010757, 1210.27013674, ..., 1239.89851747,
        1279.93925186, 1295.17952827]),
 array([1295.47133777, 1340.65409328, 1209.62393472, ..., 1241.44218266,
        1275.30536351, 1304.18238662]),
 array([1295.260345  , 1368.27500334, 1202.2864595 , ..., 1257.41873851,
        1271.03616768, 1299.78690377]),
 array([1318.81068459, 1412.19998514, 1200.16193372, ..., 1258.09359822,
        1285.84060291, 1310.06364319]),
 array([1318.96457609, 1434.12694724, 1217.75405683, ..., 1246.76672629,
        1297.73749477, 

In [51]:
np.array(cat_preds).mean(axis=0)

array([1306.4924671 , 1373.38203842, 1209.94552809, ..., 1249.20870975,
       1280.53108302, 1303.38659211])

In [54]:
test_r['Estimated_Departure_Time'] = np.array(cat_preds).mean(axis=0)
test_r.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 109019 entries, 0 to 999994
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        109019 non-null  object 
 1   Month                     109019 non-null  int64  
 2   Day_of_Month              109019 non-null  int64  
 3   Estimated_Departure_Time  109019 non-null  float64
 4   Estimated_Arrival_Time    97331 non-null   float64
 5   Cancelled                 109019 non-null  int64  
 6   Diverted                  109019 non-null  int64  
 7   Origin_Airport_ID         109019 non-null  int64  
 8   Destination_Airport_ID    109019 non-null  int64  
 9   Distance                  109019 non-null  float64
 10  Carrier_ID(DOT)           97176 non-null   float64
 11  Tail_Number               109019 non-null  int64  
 12  Delay                     27841 non-null   object 
dtypes: float64(4), int64(7), object(2)
memory us

/var/folders/4l/84htqxwn14n0jr_xvdc4tpbr0000gn/T/ipykernel_52476/1932812792.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_r['Estimated_Departure_Time'] = np.array(cat_preds).mean(axis=0)


In [56]:
train_r.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890981 entries, 0 to 890980
Data columns (total 13 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   ID                        890981 non-null  object 
 1   Month                     890981 non-null  int64  
 2   Day_of_Month              890981 non-null  int64  
 3   Estimated_Departure_Time  890981 non-null  float64
 4   Estimated_Arrival_Time    793629 non-null  float64
 5   Cancelled                 890981 non-null  int64  
 6   Diverted                  890981 non-null  int64  
 7   Origin_Airport_ID         890981 non-null  int64  
 8   Destination_Airport_ID    890981 non-null  int64  
 9   Distance                  890981 non-null  float64
 10  Carrier_ID(DOT)           793827 non-null  float64
 11  Tail_Number               890981 non-null  int64  
 12  Delay                     227160 non-null  object 
dtypes: float64(4), int64(7), object(2)
memory us

In [10]:
target = 'Delay'
features = ['Month',
 'Day_of_Month',
 'Estimated_Departure_Time',
 'Estimated_Arrival_Time',
 'Cancelled',
 'Diverted',
 'Origin_Airport_ID',
 'Destination_Airport_ID',
 'Distance',
 'Carrier_ID(DOT)',
 'Tail_Number']

train_ = train_.reset_index(drop=True)

In [11]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import GridSearchCV
from tqdm import tqdm

kf = KFold(n_splits = 10, random_state = 41, shuffle = True)
models = []
val_scores = []
preds = []

params = {
'n_estimators':5000,
'max_depth':6,
'early_stopping_rounds':200,
'learning_rate':0.01,
'one_hot_max_size':3,
'bootstrap_type':"MVS",
'l2_leaf_reg':2,
'random_state':41
         }

# bundle = [CatBoostClassifier, LGBMClassifier, XGBClassifier]
# for model in bundle:
# model = CatBoostClassifier(**params)
for i, (train_index, val_index) in tqdm(enumerate(kf.split(train_, train_[target]))):

        # clf.fit(train_x, train_y)

        X_train, X_val = train_[features].loc[train_index], train_[features].loc[val_index]
        y_train, y_val = train_[target][train_index], train_[target][val_index]

        model = CatBoostClassifier(**params)

        model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = 2000)

        models.append(model)
        score = log_loss(y_val, model.predict_proba(X_val))

        val_scores.append(score)

0it [00:00, ?it/s]

0:	learn: 0.6878246	test: 0.6878318	best: 0.6878318 (0)	total: 68ms	remaining: 5m 39s
2000:	learn: 0.4346923	test: 0.4399201	best: 0.4399201 (2000)	total: 19.8s	remaining: 29.6s
4000:	learn: 0.4256457	test: 0.4381414	best: 0.4381414 (4000)	total: 40s	remaining: 9.99s


1it [00:50, 50.87s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4379769934
bestIteration = 4781

Shrink model to first 4782 iterations.
0:	learn: 0.6878312	test: 0.6877737	best: 0.6877737 (0)	total: 11.2ms	remaining: 56.2s
2000:	learn: 0.4349094	test: 0.4379959	best: 0.4379959 (2000)	total: 20.2s	remaining: 30.2s
4000:	learn: 0.4258807	test: 0.4362211	best: 0.4362172 (3991)	total: 41.4s	remaining: 10.3s


2it [01:43, 51.83s/it]

...

4999:	learn: 0.4218350	test: 0.4396035	best: 0.4396015 (4976)	total: 49.9s	remaining: 0us

bestTest = 0.4396014641
bestIteration = 4976

Shrink model to first 4977 iterations.
0:	learn: 0.6878323	test: 0.6877875	best: 0.6877875 (0)	total: 9.92ms	remaining: 49.6s
2000:	learn: 0.4349536	test: 0.4371951	best: 0.4371951 (2000)	total: 19.6s	remaining: 29.3s
4000:	learn: 0.4260268	test: 0.4359337	best: 0.4358961 (3914)	total: 39.5s	remaining: 9.87s


9it [07:26, 47.50s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4358960655
bestIteration = 3914

Shrink model to first 3915 iterations.
0:	learn: 0.6878307	test: 0.6877816	best: 0.6877816 (0)	total: 10.5ms	remaining: 52.5s
2000:	learn: 0.4350448	test: 0.4360157	best: 0.4360157 (2000)	total: 19.8s	remaining: 29.7s
4000:	learn: 0.4261070	test: 0.4345308	best: 0.4345214 (3988)	total: 40s	remaining: 9.99s


10it [08:15, 49.55s/it]

Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.4343794712
bestIteration = 4627

Shrink model to first 4628 iterations.


In [12]:
val_scores

[0.43797699335876283,
 0.4358958149772492,
 0.43636692508490377,
 0.4353263327329921,
 0.4463164023101533,
 0.4463811067884916,
 0.4411287296930336,
 0.439601464129778,
 0.4358960654834986,
 0.43437947119816384]

In [13]:
cat_preds = []
for model in models[:10]:
    preds = model.predict_proba(test_[features])
    cat_preds.append(preds)

In [14]:
pred = np.array(cat_preds).mean(axis=0)
test_['Delay'] = [np.argmax(i) for i in pred]
test_

,ID,Month,Day_of_Month,Estimated_Departure_Time,Estimated_Arrival_Time,Cancelled,Diverted,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay
1,TRAIN_000001,8,15,740.0,1024.0,0,0,13930,14869,1250.0,20304.0,310,1
2,TRAIN_000002,9,6,1610.0,1805.0,0,0,11057,12953,544.0,19805.0,140,1
4,TRAIN_000004,1,11,900.0,1019.0,0,0,14771,10157,250.0,20304.0,556,1
7,TRAIN_000007,4,20,1815.0,1955.0,0,0,13930,13198,403.0,20304.0,173,1
9,TRAIN_000009,6,6,650.0,838.0,0,0,12992,12266,374.0,20366.0,468,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
999993,TRAIN_999993,10,17,603.0,755.0,0,0,12264,11292,1452.0,20436.0,1538,1
999995,TRAIN_999995,9,18,936.0,1243.0,0,0,13930,14100,678.0,19977.0,2477,1
999997,TRAIN_999997,6,28,800.0,1340.0,0,0,13796,12191,1642.0,19393.0,994,1
999998,TRAIN_999998,9,27,1613.0,1824.0,0,0,10693,10397,214.0,19790.0,6207,1


In [20]:
test = test.drop('Carrier_Code(IATA)',axis=1)
test = test.drop('Origin_Airport',axis=1)
test = test.drop('Destination_Airport',axis=1)
test = test.drop(['Origin_State','Destination_State'],axis=1)
test = test.drop(['Airline'],axis=1)
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 12 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   ID                        1000000 non-null  object 
 1   Month                     1000000 non-null  int64  
 2   Day_of_Month              1000000 non-null  int64  
 3   Estimated_Departure_Time  891016 non-null   float64
 4   Estimated_Arrival_Time    890952 non-null   float64
 5   Cancelled                 1000000 non-null  int64  
 6   Diverted                  1000000 non-null  int64  
 7   Origin_Airport_ID         1000000 non-null  int64  
 8   Destination_Airport_ID    1000000 non-null  int64  
 9   Distance                  1000000 non-null  float64
 10  Carrier_ID(DOT)           890994 non-null   float64
 11  Tail_Number               1000000 non-null  object 
dtypes: float64(4), int64(6), object(2)
memory usage: 91.6+ MB
